In [1]:
import sys
!{sys.executable} -m pip install neo4j==5.0.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.4/172.4 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for neo4j: filename=neo4j-5.0.1-py3-none-any.whl size=247362 sha256=1701889fb77cadc76f10ccbbc307b83f9fbf737017e691cf0452fdbaab115a6e
  Stored in directory: /home/jovyan/.cache/pip/wheels/c2/33/c6/fa22720f17810f856b71d7a5bfb579f56f6eb1fc00ba00262a
Successfully built neo4j


In [3]:
# Import the neo4j dependency
from neo4j import GraphDatabase

uri = "neo4j://corso-kg-neo4j:7687"

username = 'neo4j'
password = 'kgCefriel'
auth = (username, password)

# Create a new Driver instance
driver = GraphDatabase.driver(uri,auth=auth)
driver.verify_connectivity()

In [4]:
from os import listdir, getcwd
from os.path import abspath, basename, isfile, join, dirname

folder = dirname(getcwd())
import_path = join(folder, "data")

with driver.session() as session:
    def create_constraint_competences(tx):
        return tx.run("""
            CREATE CONSTRAINT CompetenceId IF NOT EXISTS ON (c:Competence) ASSERT c.competenceId IS UNIQUE
            """)
    def create_constraint_users(tx):
        return tx.run("""
            CREATE CONSTRAINT EmployeeId IF NOT EXISTS ON (e:Employee) ASSERT e.employeeId IS UNIQUE
            """)
    print("Create constraints")
    session.execute_write(create_constraint_competences)
    session.execute_write(create_constraint_users)

    def create_competences(tx):
        return tx.run("""
            LOAD CSV WITH HEADERS
            FROM "file:///competences.csv"
            AS row
            MERGE (c:Competence {competenceId: row.id})
            ON CREATE SET
            c.name = row.competence,
            c.description = row.description,
            c.dimension = row.dimension
            """)
    print("Create competences")
    session.execute_write(create_competences)

    def create_units(tx):
        return tx.run("""
            LOAD CSV WITH HEADERS
            FROM "file:///employees.csv"
            AS row
            MERGE (u:Unit {name: row.unit})
            """)
    print("Create units")
    session.execute_write(create_units)

    def create_employees(tx):
        return tx.run("""
            LOAD CSV WITH HEADERS
            FROM "file:///employees.csv"
            AS row
            MERGE (e:Employee {employeeId: row.id})
            ON CREATE SET
            e.name = row.name,
            e.surname = row.surname
            WITH e, row
            MATCH (u:Unit {name: row.unit})
            MERGE (e)-[r:BELONGS_TO]->(u)
            """)
    print("Create employees")
    session.execute_write(create_employees)

    def load_assignments(tx):
        cypher = """
            LOAD CSV WITH HEADERS
            FROM "file:///assignment.csv"
            AS row
            MATCH (e:Employee {employeeId: row.employee_id})
            MATCH (c:Competence {competenceId: row.competence_id})
            MERGE (e)-[r:HAS_COMPETENCE]->(c)
            ON CREATE SET r.level = row.level
        """
        return tx.run(cypher)
    print("Load assignments")
    session.execute_write(load_assignments)
    
    def load_interest(tx):
        cypher = """
            LOAD CSV WITH HEADERS
            FROM "file:///assignment.csv"
            AS row
            WITH row WHERE row.interest = "1"
            MATCH (e:Employee {employeeId: row.employee_id})
            MATCH (c:Competence {competenceId: row.competence_id})
            MERGE (e)-[r:HAS_INTEREST]->(c)
        """
        return tx.run(cypher)
    print("Load interest")
    session.execute_write(load_interest)

Create constraints
Create competences
Create units
Create employees
Load assignments
Load interest


In [5]:
def execute_query(custom_query):
    # Execute the custom query
    with driver.session() as session:
        result = session.run(custom_query)
    
        # Extract the column names and rows from the query result
        columns = result.keys()
        rows = result.data()
    
        # Print the custom query results as a table
        for col in columns:
            print(col, end="\t")
        print()
        for row in rows:
            for col in columns:
                print(row[col], end="\t")
            print()

In [20]:
# Competences of an employee
custom_query = """
    MATCH (e:Employee {name:"Jane", surname:"Brown"})-[:HAS_COMPETENCE]->(c:Competence) 
    RETURN c.competenceId AS id, c.name AS competence
"""
execute_query(custom_query)

id	competence	
2.5	Netiquette	
3.3	Copyright and licences	
4.2	Protecting personal data and privacy	
5.3	Creatively using digital technologies	
2.1	Interacting through digital technologies	
1.3	Managing data, information and digital content	
4.4	Protecting the environment	
5.1	Solving technical problems	
4.3	Protecting health and well-being	
3.4	Programming	


In [21]:
# Employees with a given competence
competence = "3.4"
custom_query = f"""
    MATCH (e:Employee)-[:HAS_COMPETENCE]->(c:Competence {{ competenceId : '{competence}'}}) 
    RETURN e.name, e.surname
"""
execute_query(custom_query)

e.name	e.surname	
Olivia	Wilson	
David	Brown	
Jane	Taylor	
David	Davis	
Emma	Johnson	
David	Miller	
Michael	Wilson	
Jane	Davis	
David	Harris	
David	Taylor	
Daniel	Brown	
Emma	Clark	
Sophia	Johnson	
Sophia	Harris	
Jane	Brown	
Michael	Johnson	
David	Miller	


In [22]:
# Employees having the competences of an employee
custom_query = """
    MATCH (e:Employee {name:"Jane", surname:"Brown"})-[:HAS_COMPETENCE]->(c:Competence)<-[:HAS_COMPETENCE]-(oe:Employee) 
    RETURN c.name, collect(oe.employeeId)
"""
execute_query(custom_query)

c.name	collect(oe.employeeId)	
Netiquette	['EMPLOYEE_19', 'EMPLOYEE_12', 'EMPLOYEE_20', 'EMPLOYEE_21', 'EMPLOYEE_4', 'EMPLOYEE_9', 'EMPLOYEE_13', 'EMPLOYEE_6', 'EMPLOYEE_16', 'EMPLOYEE_8', 'EMPLOYEE_15', 'EMPLOYEE_23', 'EMPLOYEE_10', 'EMPLOYEE_22', 'EMPLOYEE_5', 'EMPLOYEE_1']	
Copyright and licences	['EMPLOYEE_16', 'EMPLOYEE_22', 'EMPLOYEE_2', 'EMPLOYEE_13', 'EMPLOYEE_4', 'EMPLOYEE_27', 'EMPLOYEE_29', 'EMPLOYEE_30', 'EMPLOYEE_20', 'EMPLOYEE_6', 'EMPLOYEE_28', 'EMPLOYEE_8', 'EMPLOYEE_23', 'EMPLOYEE_3']	
Protecting personal data and privacy	['EMPLOYEE_17', 'EMPLOYEE_16', 'EMPLOYEE_23', 'EMPLOYEE_10', 'EMPLOYEE_2', 'EMPLOYEE_19', 'EMPLOYEE_30', 'EMPLOYEE_22', 'EMPLOYEE_8', 'EMPLOYEE_3', 'EMPLOYEE_29', 'EMPLOYEE_4', 'EMPLOYEE_15', 'EMPLOYEE_20']	
Creatively using digital technologies	['EMPLOYEE_6', 'EMPLOYEE_16', 'EMPLOYEE_21', 'EMPLOYEE_3', 'EMPLOYEE_22', 'EMPLOYEE_30', 'EMPLOYEE_23', 'EMPLOYEE_10', 'EMPLOYEE_8', 'EMPLOYEE_19', 'EMPLOYEE_12', 'EMPLOYEE_2', 'EMPLOYEE_29', 'EMPLOYEE_4', 'EM

In [19]:
# More "similar" employees
custom_query = """
    MATCH (e:Employee {name:"Jane", surname:"Brown"})-[:HAS_COMPETENCE]->(c:Competence)<-[:HAS_COMPETENCE]-(oe:Employee)
    WITH e, oe, COLLECT(c) as competences
    RETURN oe.name AS name, oe.surname AS surname, size(competences) as num_common_competences ORDER BY num_common_competences DESC
"""
execute_query(custom_query)

name	surname	num_common_competences	
David	Brown	10	
Jane	Taylor	10	
David	Davis	9	
David	Miller	9	
Sophia	Johnson	9	
Jane	Davis	8	
Daniel	Brown	8	
David	Harris	7	
David	Johnson	7	
Sophia	Harris	7	
Michael	Wilson	7	
David	Brown	7	
Emma	Johnson	6	
Olivia	Davis	6	
Andrew	Wilson	5	
Daniel	Davis	5	
Emma	Clark	5	
Olivia	Clark	4	
Sophia	Davis	4	
Michael	Johnson	4	
David	Miller	4	
David	Taylor	3	
Jane	Smith	3	
Olivia	Wilson	2	
Emily	Wilson	1	
Emily	Brown	1	


In [ ]:
# Close the Neo4j driver
driver.close()